## 多元线性回归
- 对于一个有n个特征的样本i而言，它的回归结果如下方程：$$\hat{y}=\omega _0+\omega _1x_{i1}+\omega _2x_{i2}+\dots+\omega _nx_{in}$$
    - $\omega$被称为模型的参数，其中$\omega _0$被称为截距，$\omega _0$~$\omega _n$被称为回归系数，有时也用$\theta$表示。$y$是目标变量，$x_{i1}$~$x_{in}$是样本上的不同特征。
- 我们使用矩阵来表示回归方程，$Y$是包含了$m$个样本的回归结果的列向量，其中$W$可以看做是一个长度为$n$的列相量，$X$是一个结构为$(m,n)$的特征矩阵 
$$\left[\begin{matrix} \hat{y}_0 \\ \hat{y}_1 \\ \hat{y}_2 \\ \dots \\ \hat{y}_m \end{matrix}\right] = 
\left[\begin{matrix} 1 & 1 & 1 & \dots & 1 \\ 
x_{11} & x_{12} & x_{13} & \dots & x_{1n} \\
x_{21} & x_{22} & x_{23} & \dots & x_{2n} \\
\dots & \dots & \dots & \dots & \dots\\
x_{m1} & x_{m2} & x_{m3} & \dots & x_{mn} \\
\end{matrix}\right] * 
\left[\begin{matrix} \omega _0 \\ \omega _1 \\ \omega _2 \\ \dots \\ \omega _m \end{matrix}\right]$$
$$ Y = XW$$
- 线性回归的任务是构造一个预测函数来映射输入的特征矩阵$X$和标签值$Y$的线性关系，而构造预测模型的核心就是找出模型的参数向量$W$。
    - 在多元线性回归中，我们将损失函数定义为：$$\sum^m_{i=1}(y_i-\hat{y}_i)^2=\sum^m_{i=1}(y_i-X_iW)^2$$
    - 我们的目标就变成求 $min (||Y-XW||_2)^2$，即SSE(Sum of Squared Error,误差平方和) 或者 RSS(Residual Sum of Squares,残差平方和)。
    - 最优解为$W=(X^TX)^{-1}X^TY$
- 模型评估指标
    - MSE(Mean Squared Error)均方误差 $$MSE=\frac{\sum^m_{i=1}(y_i-\hat{y}_i)^2}{m}$$
    - MAE(Mean Absolute Error)绝对均值误差 $$MAE=\frac{\sum^m_{i=1}|y_i-\hat{y}_i|}{m}$$
    - $R^2\text{和}EVS$(Explained Variance Score,可解释性方差分数)用来衡量模型是否较好的拟合了数据的规律(数据的分布规律，单调性等) $$R^2=1-\frac{\sum^m_{i=0}(y_i-\hat{y}_i)^2}{\sum^m_{i=0}(y_i-\bar{y})^2}=1-\frac{RSS}{\sum^m_{i=0}(y_i-\bar{y})^2}$$ $$EVS=1-\frac{Var(y_i-\hat{y})}{Var(y_i)}$$
        - 其中，Var()表示方差

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 支持显示中文
plt.rcParams['font.family'] = 'Heiti TC'
plt.rcParams['axes.unicode_minus']=False
sns.set_theme(font='Heiti TC')

In [ ]:
data = pd.read_csv('milk_power.csv')
data.head()

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data['段位'].describe()

In [ ]:
pd.crosstab(data['段位'], [data['分类'], data['配方']], margins=True)

In [ ]:
df = data.groupby('段位').size()
df.plot(kind='pie', subplots=True, figsize=(6,6), autopct='%1.1f%%', fontsize=20, colormap='Set3')
plt.legend(loc='upper right')
plt.show()

In [ ]:
area = data.groupby('奶源产地').size().sort_values()
sns.barplot(area, palette='Set3')

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn import linear_model, preprocessing

In [ ]:
data.columns

In [ ]:
data1 = {"y": data['评价量'], "x1": data['团购价.元.'], "x2": data['商品毛重.kg.']}
df = pd.DataFrame(data1)
df.head()

方法一：

In [ ]:
Y = df['y']
X = df[['x1', 'x2']]

In [ ]:
X = sm.add_constant(X)
model2 = sm.OLS(Y, X)
model2 = model2.fit()

方法二：

$\text{原假设}H_0:\beta _1=\beta _2=\dots=\beta _k=0$

In [ ]:
model1 = smf.ols('y ~ 1 + x1 + x2', df)
result1 = model1.fit()
result1.summary()

In [ ]:
enc = preprocessing.OneHotEncoder()

In [ ]:
enc.fit_transform(np.array(data['段位']).reshape(-1,1)).toarray()

In [ ]:
# 创建虚拟变量
factor = pd.get_dummies(data['段位'])
factor.insert(0, '评价量', data['评价量'])
factor.head()

In [ ]:
y = factor['评价量']
x1 = factor['1段']
x2 = factor['2段']
x3 = factor['3段']
x4 = factor['4段']

In [ ]:
model2 = smf.ols('y ~ 1 + x1 + x2 + x3', factor)
result2 = model2.fit()
result2.summary()

In [ ]:
data

In [ ]:
X = data['商品名称'].to_frame()
for i in range(data.shape[1]):
    j = 0
    if i!=0 and i!=1 and i!=9 and i!=10:
        X = pd.concat([X, pd.get_dummies(data.iloc[:, i])], axis=1)
        j += 1
        
X = pd.concat([X, data['商品毛重.kg.'], data['团购价.元.']], axis=1)
X.drop(columns='商品名称', inplace=True)
X = np.array(X, dtype=float)

In [ ]:
y = np.array(data['评价量'], dtype=float)

In [ ]:
X = sm.add_constant(X)
mod = sm.OLS(y, X)
res = mod.fit()

In [ ]:
res.summary()

In [ ]:
for i in range(30):
    print(variance_inflation_factor(X,i))